In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist
import math
import numpy as np

##### Set high-level variables

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS            
flags.DEFINE_string('train_dir', 'data', 'Directory to put the training data.')
flags.DEFINE_boolean('fake_data', False, 'If true, uses fake data for unit testing.')

In [8]:
print(type(FLAGS))

<class 'tensorflow.python.platform.default._flags._FlagValues'>


In [16]:
batch_size=128
IMAGE_PIXELS=28*28
hidden1_units=1024
hidden2_units=1024
NUM_CLASSES=10
learning_rate = 0.5
max_steps = 3001

##### Extract data in needed formats

In [3]:
data_sets = input_data.read_data_sets(FLAGS.train_dir, FLAGS.fake_data)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
data_sets.train.labels[138]

4

In [22]:
def reformat_label(labels):
  # dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(NUM_CLASSES) == labels[:,None]).astype(np.float32)
  return labels

# Datasets are Nx724
train_dataset = data_sets.train.images
valid_dataset = data_sets.validation.images
test_dataset = data_sets.test.images

# Labels are (N,10) (1hot encoded)
train_labels = reformat_label(data_sets.train.labels)
valid_labels = reformat_label(data_sets.validation.labels)
test_labels = reformat_label(data_sets.test.labels)

# Same labels are (N,) (NOT 1hot encoded)
train_labels_int = data_sets.train.labels
valid_labels_int = data_sets.validation.labels
test_labels_int = data_sets.test.labels

print('Training set', train_dataset.shape, train_labels.shape, train_labels_int.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape, valid_labels_int.shape)
print('Test set', test_dataset.shape, test_labels.shape, test_labels_int.shape)

('Training set', (55000, 784), (55000, 10), (55000,))
('Validation set', (5000, 784), (5000, 10), (5000,))
('Test set', (10000, 784), (10000, 10), (10000,))


##### Build the Graph
Questions:  
* ** eval_correct: ** Set up eval_correct tensor to call in order to periodically evalulate againstt my validation set.  What gets execcuted in the graph when I do this?  Is it just simply comparing the calculated logits against the responses?  Or is it now using the validated set to continue training the graph?
* ** tensor states: ** How to peer into each tensor to view it's values?  How to construct breakpoints in the midst of processing to look at individual tensors.
* ** train_prediction: ** I created this when running the training data thru the loop and then calculating the prediction based on weights.  However if I run my validation sets thru this, am I calculating a train_prediction as well with the validation set?  How does the graph know not to adjust the weights for this round?
* **placeholder_labels and placeholder_labels_int: ** kludge-y
* **eval section:** When I use these tensors to check accuracy across entire test set, I noticed that it does not run the logits through the softmax.  Shouldn't it?  It seems that the predictions should always go thru the Softmax?


In [102]:
graphn = tf.Graph()

with graphn.as_default():
    # Inputs and PLACEHOLDERS
    # Originally set the shape for each of these to specifically use batch_size for # rows
    # This caused problems when I wanted to run the entire test set to get an accuracy score
    # because batch_size was 128 and the test set dimension was much larger (10000, 784)
    # then I saw that the tutorial used None instead and that worked for everything
    images_placeholder     =tf.placeholder(tf.float32, shape=(None, IMAGE_PIXELS))
    labels_placeholder     =tf.placeholder(tf.float32, shape=(None, NUM_CLASSES))
    labels_placeholder_int =tf.placeholder(tf.int32, shape=(None))  
   
    # INFERENCE
    with tf.name_scope('hidden1') as scope:
        weights = tf.Variable(tf.truncated_normal([IMAGE_PIXELS, hidden1_units], 
            stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))), name='weights')
        biases = tf.Variable(tf.zeros([hidden1_units]), name = 'biases')
        hidden1 = tf.nn.relu(tf.matmul(images_placeholder, weights) + biases)

    with tf.name_scope('hidden2') as scope:
        weights = tf.Variable(tf.truncated_normal([hidden1_units, hidden2_units],
            stddev=1.0 / math.sqrt(float(hidden1_units))), name='weights')
        biases = tf.Variable(tf.zeros([hidden2_units]), name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)

    with tf.name_scope('softmax_linear') as scope:
        weights = tf.Variable(tf.truncated_normal([hidden2_units, NUM_CLASSES],
            stddev=1.0 / math.sqrt(float(hidden2_units))),name='weights')
        biases = tf.Variable(tf.zeros([NUM_CLASSES]),name='biases')
        logits = tf.matmul(hidden2, weights) + biases
      
    # LOSS
    # softmax_cross_entropy needs labels_placeholder to be float32 1hot encoded
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels_placeholder))
    
    # TRAINING
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = optimizer.minimize(loss, global_step=global_step)
    train_prediction = tf.nn.softmax(logits)
    
    # EVAL (use either eval_acc op or the eval_correct op)
    # Gets the index of the largest value across the tensor, test_a is prediction, test_b is response
    test_a = tf.argmax(logits,1)
    test_b = tf.argmax(labels_placeholder,1)
    # Boolean tensor
    correct_prediction = tf.equal(test_a, test_b)
    cast_float = tf.cast(correct_prediction, tf.float32)
    # Get mean across tensor, ie sum/length
    eval_acc = tf.reduce_mean(cast_float)
    # in_top_k requires labels_placeholder to be int32 or int64 list
    correct = tf.nn.in_top_k(logits, labels_placeholder_int, 1)
    eval_correct = tf.reduce_sum(tf.cast(correct, tf.int32))

In [15]:
print(labels_placeholder_int.get_shape, labels_placeholder.get_shape)

(<bound method Tensor.get_shape of <tf.Tensor 'Placeholder_2:0' shape=(128,) dtype=int32>>, <bound method Tensor.get_shape of <tf.Tensor 'Placeholder_1:0' shape=(128, 10) dtype=float32>>)


In [39]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

##### Training Loop
Questions:
* **feed_dict:** See a) below. What does it mean if I don't include one of the placeholders in the graph?  Seems like that tensor just does not get called?  If I call that tensor, I have to includ the right placeholder or it chokes
* ** Mini batches:** When it is appropriate to do mini-batches?  Is that just for training?  What if I just want to simply predict (and evaluate)?  Why do I need to do this as a mini-batch?
* **train vs. test**: How does the graph know when you are training it to update weights, vs just feeding it a data set to calculate accuracy (and not change weights or bias values)?  Would you ever want to push the test data into smaller batch sizes?  In the exmaple below, I did not do that... Is it that if you don't call the train_op vector which tries to minimize loss which would then update the weights and biases?  But when I removed that from my final test run, I did not notice any difference in errors from when I did call for loss.

In [106]:
# TRAINING LOOP

with tf.Session(graph=graphn) as session:
    init = tf.initialize_all_variables().run()
    sess = tf.Session()
    
    for step in xrange(max_steps):
        # Generates the starting index from train_dataset to extract the minibatch
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        batch_labels_int = train_labels_int[offset:(offset + batch_size)]
        # A) The fact that I only feed it labels_placeholder and not labels_placeholder_int?
        feed_dict = {images_placeholder : batch_data, labels_placeholder : batch_labels, labels_placeholder_int : batch_labels_int}
        _, l, predictions, num_correct = session.run([train_op, loss, train_prediction, eval_correct], feed_dict=feed_dict)       
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            # Evaluate the entire dataset in specified batch-size increments, return all predictions
            acc, eval_predictions = do_eval(valid_dataset, valid_labels, valid_labels_int, batch_size, 
                    images_placeholder, labels_placeholder, labels_placeholder_int)
            print("Validation accuracy: %4.6f" %(acc))
    
    fdict = {images_placeholder : test_dataset, 
             labels_placeholder : test_labels,
             labels_placeholder_int : test_labels_int}
    logits, eval_correct, test_acc = session.run([logits, eval_correct, eval_acc], feed_dict=fdict)
    print('Test dataset accuracy %4.2f%%' %(100*eval_correct/float(len(test_dataset))))
    
    #print(sess.run(logits, feed_dict={images_placeholder: test_dataset,labels_placeholder: test_labels}))
            

Minibatch loss at step 0: 2.299736
Minibatch accuracy: 14.1%
Validation accuracy: 26.970167
Minibatch loss at step 500: 0.153297
Minibatch accuracy: 97.7%
Validation accuracy: 96.803245
Minibatch loss at step 1000: 0.019522
Minibatch accuracy: 100.0%
Validation accuracy: 97.597866
Minibatch loss at step 1500: 0.054390
Minibatch accuracy: 97.7%
Validation accuracy: 97.580363
Minibatch loss at step 2000: 0.054306
Minibatch accuracy: 98.4%
Validation accuracy: 97.302871
Minibatch loss at step 2500: 0.009856
Minibatch accuracy: 100.0%
Validation accuracy: 98.086604
Minibatch loss at step 3000: 0.002164
Minibatch accuracy: 100.0%
Validation accuracy: 98.289689
Test dataset accuracy 0.01%


array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.], dtype=float32)

In [25]:
def do_eval(valid_dataset, valid_labels, valid_labels_int, batch_size, 
            images_placeholder, labels_placeholder, labels_placeholder_int):
    true_count = 0  # Counts the number of correct predictions.
    steps_per_epoch = valid_dataset.shape[0] // batch_size
    num_examples = steps_per_epoch * batch_size
    eval_predictions = []
    for step in xrange(steps_per_epoch):
        eval_offset = (step * batch_size) % (valid_labels.shape[0] - batch_size)
        eval_batch_data = valid_dataset[eval_offset:(eval_offset + batch_size), :]
        eval_batch_labels = valid_labels[eval_offset:(eval_offset + batch_size), :]
        eval_batch_labels_int = valid_labels_int[eval_offset:(eval_offset + batch_size)]
        f_dict = {images_placeholder : eval_batch_data, labels_placeholder : eval_batch_labels,
                  labels_placeholder_int : eval_batch_labels_int}
        eval_predictions += session.run([eval_correct, loss, train_op], feed_dict=f_dict)
    #print("validation accuracy: %4.6f%%" %(100*sum(eval_predictions)/float(num_examples)))    
    return 100*sum(eval_predictions)/float(num_examples), eval_predictions

In [ ]:
labels_placeholder_int